### Исследование по работе с аудиофайлами

---
##### Постановка задачи

Проверить, как хорошо `google cloud speech-to-text api` распознает аудиофайлы с английской речью

---
##### Идея

Пропустим через апи песни Эда Ширана, надеясь получить внятный ответ

Если апи будет сложно переварить песни, пропустим несколько минут аудиокниги

---
##### Реализация

- Была произведена перекодировка `mp3` файлов в `wav`
- Был создан аккаунт на `google cloud`
- Были подключены все сервисы включая саму `api` и `bucket`
- Оснавная функция загружает в `bucket` аудиофайл, транскрибирует его на серверах google, выгружает обратно и удаляет файл из бакета

Была произведена перекодировка `mp3` файлов в `wav`

```cpp
for song in songs:
    sound = AudioSegment.from_mp3(song)
    sound = sound.set_channels(1)
    sound.export(song, format="wav")
```

Загрузка файлов на сервер

```cpp
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client.from_service_account_json('acc.json')
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)
```

Удаление файлов с сервера

```cpp
def delete_blob(bucket_name, blob_name):
    storage_client = storage.Client.from_service_account_json('acc.json')
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    blob.delete()
```

Обработка на сервере и выгрузка

```cpp
client = speech.SpeechClient.from_service_account_json('acc.json')
audio = types.RecognitionAudio(uri=gcs_uri)

config = types.RecognitionConfig(
    encoding = speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=frame_rate,
    language_code='en-US')

operation = client.long_running_recognize(request={"config": config, "audio": audio})
response = operation.result(timeout=10000)

for result in response.results:
    transcript += result.alternatives[0].transcript

delete_blob(bucket_name, destination_blob_name)
return transcript
```

---
##### Результаты

К сожалению, результаты транскрипции песен оставляют желать лучшего:

`First Times - Ed Sheeran lyrics:`

I thought it'd feel different playin' Wembley<br />Eighty thousand singin' with me<br />It's what I've been chasin' 'cause this is the dream<br />When it was all over, I cleared out the room<br />Grabbed a couple beers, just me and you<br />Then we started talking the way that we do<br />Ain't it funny how the simplest things in life can make a man?<br />Little moments that pass us by<br />Oh, but I remember<br />The first kiss, the first...

`transcription:`

go to feel different singing with me but I've been chasing this is the dream when it was just me and you doing San Antonio simple things in life can make a little more...

`transcription (vocals only):`

go to feel different playing wimbley 80000 singing with me sweat up and chasing this is the dream when it was sold over grabbed a couple days just me and you then we start talking San Antonio them things left me little more

А вот результаты обработки аудиокниги наоборот удивляют:

real text|transcription|
:---:|:---:| 
spiritual; secondly, that of the sense of physical,existence. It seems probable that if, upon reaching the second stage, we could recall the impressions of the first, we should find these impressions eloquent in memories of the **gulf** beyond. And that **gulf** is—what? How at least shall we distinguish its shadows from those of the tomb? But if the impressions of what I have termed the first **stage are** not, at will, recalled, yet, after long interval, do they not come unbidden, while we marvel **whence** they come? He who has never swooned, is not he who finds strange palaces and wildly familiar faces **in coals** that glow; is not he who beholds floating in mid-air **the sad** visions that the many may not view; is not he who ponders over the perfume of some novel flower; is|spiritual secondly that of the sense of physical existence it seems probable that if upon reaching the second stage we could recall the impressions of the first we should find these Impressions eloquent in memories of the **golf** Beyond and that **golf** is what how at least shall we distinguish its Shadows from those of the Tomb but if the impressions of what I have termed the first **state I'm** not at will recalled yet after a long interval do they not come unbidden while we Marvel **Wednesday** come he who has never swooned is not he who finds strange palaces and wildly familiar faces **and Cole's** that glow is not he who beholds floating in mid-air **beside** visions that the many may not view it's not he who Ponders over the perfume of some novel flower is|
I WAS sick—sick unto death with that long agony; and when they at length unbound me, and I was permitted to sit, I felt that my senses were leaving me. The sentence—the dread sentence of death—was the last of distinct accentuation which reached my ears. After that, the sound of the inquisitorial **voices seemed merged** in one dreamy indeterminate hum. It conveyed to my soul the idea of revolution—perhaps from its association **in** fancy with the **burr** of a mill-wheel. This only for a brief period, for presently I heard no more. Yet, for a while, I saw—**but** with how terrible an exaggeration!|I was sick sick unto death with that long Agony and when they at length Unbound me and I was permitted to sit I felt that my senses were leaving me the sentence The Dread sentence of death was the last of distinct accentuation which reached my ears after that the sound of the inquisitorial **voice emerged** in one dreamy indeterminate hum it conveyed to my soul the idea of Revolution perhaps from its Association **and** fancy with the **borough** of a mill-wheel this only for a brief period for presently I heard no more yet for a while I saw **and** with how terrible and exaggerated|

---
##### Выводы

Такой результат может быть связан со спецификой записи данных аудио файлов. Аудиокнига записывается специальным актёром, а над текстом работает профессиональный редактор. Он расставляет ударения, выписывает слова, где не очевидное для него ударение, выписывает термины, географические названия, имена собственные. После записи книги, весь материал отдаётся монтажёру, который выстраивает паузы, вырезает посторонние звуки, присвисты и другие посторонние шумы. Неисправные дубли, в которых слова произнесены нечётко или неверно, будут переписаны. В песне, напротив, многие слова могут быть изменены для сохранения рифмы и темпа музыки, а паузы между словами могут быть не очевидны. При этом певец пользуется всем имеющимся у него диапазоном голоса, а не его средним отрезком. Также настолько разные результаты могли возникнуть из-за разного диалекта диктора и певца, и хотя документация говорит о том, что Механизм распознавания речи в текст поддерживает множество языков и диалектов, всё же предпочтительным является Американский английский, на котором записана аудиокнига, песню же исполняет певец с британским акцентом, что, очевидно, сказалось на обработке. Всё это позволяет сделать вывод о том, что для распознавания аудиокниги были созданы более приятные условия, чем для распознавания текста песни, поэтому и результат распознавания аудиокниги получился наиболее удачным